<a href="https://colab.research.google.com/github/PigStep/CIFAR-10-based-Content-categorizator/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this notebook about
This notebook tracks experiments, code and data manipulation for model

In [ ]:
import torchvision
import torch
from torchvision.transforms import transforms

In [ ]:
batch_size = 4

transform = transforms.ToTensor()

#Getting CIFAR-10 dataset with batches
trainset = torchvision.datasets.CIFAR10(root="/content",train=True,transform=transform,download=True)
trainloader = torch.utils.data.DataLoader(dataset = trainset,batch_size=batch_size,shuffle=True)

testset = torchvision.datasets.CIFAR10(root="/content",train=False,transform=transform,download=True)
testloader = torch.utils.data.DataLoader(dataset = testset,batch_size=batch_size,shuffle=True)

In [ ]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /content
    Split: Train
    StandardTransform
Transform: ToTensor()

# Baseline model: self-coded CNN

In [ ]:
import numpy as np
class baseCNN:
  def __init__(self):
    pass

  def convolutinal_layer(image,filters,stride=1):
    """
    Convolves image with filters.
    params:
      `image` - image as numpy array
      `filter` - collection of filters as numpy array
      `stride` - stride for filter window
    """
    (image_H,image_W,image_channels) = image.shape
    (num_filters, filter_H, filter_W, filter_channels) = filters.shape

    out_H = (image_H - filter_H) // stride +1
    out_W = (image_W - filter_W) // stride +1
    feature_map = np.zeros((out_H,out_W,num_filters))

    for y in range(0,image_H - filter_H + 1, stride):
      for x in range(0,image_W - filter_W + 1, stride):
        # Get filter slice of image
        filter_slice = image[y:y+filter_H, x:x+filter_W, :]

        for i, filter in enumerate(filters):
          # Get matrix features for every channels
          filter_matrix_channels = np.sum(filter * filter_slice)
          # Save filter matrix features at coordinates position
          feature_map[y // stride, x // stride, i] = filter_matrix_channels

    return feature_map

    def pooling_layer(features_maps,pooling_size=2,stride=2):
      """
      Max pooling images

      params:
        `feature_maps` - maps of the features
        `pooling_size` - size of the window
        `stride` - stride of the window
      """
      (feature_map_H,feature_map_W,num_filters) = features_maps.shape

      out_H = (feature_map_H - pooling_size) // stride +1
      out_W = (feature_map_W - pooling_size) // stride +1

      pooling_map = np.zeros((out_H,out_W,num_filters))

      for y in range(0,(feature_map_H - pooling_size)//stride +1, stride):
        for x in range(0,(featuremap_W - pooling_size)//stride +1, stride):
          for i in range(num_filters):
            feature_window = feature_map[y:y+pooling_size,x:x+pooling_size, i]
            pooling_map[y // stride,x // stride,i] = np.max(feature_window)

      return pooling_map

    def fully_connected_layer(pooling_map,weights,bias):
      """
      Fully connected layer for weighted sum

      params:
        `pooling_map` - map of the features
        `weights` - weights of the layer
        `bias` - bias of the layer
      """
      vector = pooling_map.flatten()
      return np.dot(vector,weights) + bias

      def softmax(x):
        return np.exp(x) / np.sum(np.exp(x))

      def cross_entropy(y_pred,y_true):
        return - np.sum(y_true * np.log(y_pred))

      def dense_backprop(dL_dz, x, weights, bias, learning_rate = 0.1):
        """
        Backpropagation for dense layer
        params:
          `dL_dz` - derivative of the loss with respect to the output of the layer
          `x` - input
          `weights` - weights of the layer
          `bias` - bias of the layer
          `learning_rate` - learning rate for gradient descent
        """

        dL_dw = dL_dz * x
        dL_db = dL_dz
        dL_dx = dL_dz * weights

        weights = weights - learning_rate * dL_dw
        bias = bias - learning_rate * dL_db

        return weights, bias, dL_dx

      def pooling_backprob(dl_dp,x,pooling_size=2,stride=2):
        """
        Backpropagation for pooling layer
        params:
          `dl_dp` - derivative of the loss with respect to the output of the layer
          `x` - input to pooling layer
          `pooling_size` - size of the window
          `stride` - stride of the window
        """
        (in_H, in_W, in_C) = x.shape
        dL_dx = np.zeros(x.shape)

        for y in range(0, (in_H - pooling_size) // stride +1, stride):
          for x in range(0, (in_W - pooling_size) // stride +1, stride):
            for c in range(in_C):
              feature_window = x[y:y+pooling_size, x:x+pooling_size, c]

              max_index = np.argmax(feature_window)
              (max_y,max_x) = max_index // pooling_size, max_index % pooling_size

              # Fill previous masked points by derrivative
              dL_dx[y + max_y, x + max_x, c] = dl_dp[y // stride, x // stride, c]

        return dL_dx